In [10]:
import pandas as pd
import numpy as np
import os

dataset_folder = '../HSPP Data'

In [11]:
def create_notable_groups(df):
    #Keyword/phrase lists
    client_death = ["Death", "Deceased"]
    client_hospitalization = ["Hospitalization", "Admitted to Hospital", "In Hospital", "Health/Hospital", 
        "hospital", "hospital", "hospitalization", "admitted to hospital", "In hospital"]
    client_fall = ["Injury/Fall", "Injury", "Near Fall (stabilized or assisted down)", "Close call - fall",
        "Fell to Floor (witnessed)", "Found on Floor (unwitnessed)", "Client/Family Reported Fall", 
        "client fall and change of hours needed", "caregiver reported client had a fall", "fall", "Fall", "Client had a fall and went by ambulance to hospital",
        "Fell onto couch did not reach floor", "fall/incident"]
    client_health_decline = ["Health Decline", "Cognitive/Mental Health Decline", "Physical Health Decline", 
        "Cognitive Health Decline", "Visit to ER", "Phyical Health Decline", "Memory decline", 
        "Trip to hospital/client unresponsive", "Health Status Decline", "Visit to ER", "Physical Health Concerns", 
        "taken by ambulance to hospita;l", "Multiple Suicide Attempts", "client went by ambulance to the hospital", "Client taken to hospital", 
        "Mobility decline", "Decreased balance over time, refusal to use assistive devices properly", "Health decline concern"]
    client_account_status = ["Account Activated/Reactivated", "Service Terminated", "Service Suspended", "suspended services", 
        "Suspend services", "Resuming services", "Termination Of Services", "Termination of Service", "Suspending services", "Admitted to Hospice Care",
        "Service Cancellation", "suspended services COVID-19", "Cancelation", "Cancellation of services", "cancelling services....", "Cancellation of services"]
    client_potential_LOC_change_health = ["Behaviour", "behavior", "Client Behavior", "client behaviour", "Behavior", 
        "behavioural concern","Client behavior", "Client Behavior Concern", "Behavior Issue", "Client behavior concerns", "New Diagnosis", "Reassessment", 
        "Health/Cognitive Update", "Returned from Hospital", "reassessment", "re-assessment", "Seizure", "seizure", 
        "ambulance called", "client discharged", "first aid needed", "Re-assessment", "Mental Health Concern", "HEALTH CONCERN",
        "health concern", "Health Concern", "Change in health", "Significant change", "Significant Change",
        "Near Choking Incident", "Choking", "Health Concern call to POA", "Health Concerns", "Feeling unwell/ High blood sugar", 
        "Health Change", "High Blood Pressure", "Urinary tract infection", "Urinary Tract Infections", "Client has high blood pressure and had swollen ankle",
        "Client is not happy", "Chest Pain", "Client in Pain", "car accident", "suicidal actions", "health and memory concerns", "Feeling Unwell", 
        "Health Concern/ Support Concern", "Low Blood Sugar - Ambulance called", "Health Status Dramatic Change", "Concern about wandering", "Confused Client",
        "Unsafe Steps"]
    client_plan_update = ["Environment/Scheduling Preferences Update", "Care Plan Update", "Scheduling Issue"]
    client_health_status = ["Health Status", "Update", "Assessment", "health status", "Physically Health Update", "Physical Health Update", 
        "Health Status Update", "Health Update", "Physical Update", "Overall Health Status", "Mental Health Status"]
    client_covid = ["COVID-19 CASE"]

    notable_type = []
    # Look at notables in order of importance for the type of label to our analyses 
    # so that we put each notable in the most appropriate (i.e. interesting) group:
    # deaths > 
    #   hospitalizations > 
    #       injuries/falls > 
    #           health decline > 
    #               account status change > 
    #                   covid > 
    #                       Level of care (LOC) change > 
    #                           Potential LOC change (health) > 
    #                               potential LOC change (non-health) > 
    #                                   Client Issues/Incidents/Complaint > 
    #                                       Care plan/Scheduling update Health Status Check > 
    #                                           Other
    for i in df.index:
        # Step 1: deaths
        if (df['notable_type'][i] in client_death) or \
            (df['record_type'][i] in client_death) or \
            (df['nature_of_incident'][i] in client_death) or \
            (df['reason'][i] in client_death):
            notable_type.append([df['notableId'][i], "Death"])

        # Step 2: hospitalizations
        elif (df['notable_type'][i] in client_hospitalization) or \
            (df['record_type'][i] in client_hospitalization) or \
            (df['nature_of_incident'][i] in client_hospitalization) or \
            (df['reason'][i] in client_hospitalization):
            notable_type.append([df['notableId'][i], "Hospitalization"])

        # Step 3: injuries and falls
        elif (df['notable_type'][i] in client_fall) or \
            (df['record_type'][i] in client_fall) or \
            (df['nature_of_incident'][i] in client_fall) or \
            (df['reason'][i] in client_fall):
            notable_type.append([df['notableId'][i], "Injury/Fall"]) 

        # Step 4: Health decline
        elif (df['notable_type'][i] in client_health_decline) or \
            (df['record_type'][i] in client_health_decline) or \
            (df['nature_of_incident'][i] in client_health_decline) or \
            (df['reason'][i] in client_health_decline):
            notable_type.append([df['notableId'][i], "Health Decline"])

        # Step 5: Account status change
        elif (df['notable_type'][i] in client_account_status ) or \
            (df['record_type'][i] in client_account_status ) or \
            (df['nature_of_incident'][i] in client_account_status ) or \
            (df['reason'][i] in client_account_status ):
            notable_type.append([df['notableId'][i], "Account Status Change"]) 

        # Step 5: got CoVID
        elif (df['notable_type'][i] in client_covid):
            notable_type.append([df['notableId'][i], "CoVID 19"])

        # Step 6: Level of care (LOC) change (but no details on level up or down provided)
        elif (df['record_type'][i] == "Level_of_Care_Change"):
            notable_type.append([df['notableId'][i], "LOC Change"]) 

        # Step 7: Potential for LOC change due to health changes
        elif (df['notable_type'][i] in client_potential_LOC_change_health ) or \
            (df['record_type'][i] in client_potential_LOC_change_health ) or \
            (df['nature_of_incident'][i] in client_potential_LOC_change_health ) or \
            (df['reason'][i] in client_potential_LOC_change_health ):
            notable_type.append([df['notableId'][i], "Potential LOC Change (Health)"]) 

        # Step 8: Potential for LOC change due to non-health changes
        elif (df['notable_type'][i] == "Change in Living Arrangements") or \
            (df['notable_type'][i] == "Family Support/Problems"):
            notable_type.append([df['notableId'][i], "Potential LOC Change (Non-Health)"])   

        # Step 9: Client Issues/Incidents/Complaint
        elif (df['notable_type'][i] == "Client Incident") or \
            (df['notable_type'][i] == "Client Complaint") or \
            (df['notable_type'][i] == "Client Issue"):
            notable_type.append([df['notableId'][i], "Client Issue/Incident/Complaint"]) 

        #Step 10: Care plan/Scheduling update 
        elif (df['notable_type'][i] in client_plan_update):
            notable_type.append([df['notableId'][i], "Care Plan/Schedule Update"]) 

        # Step 11: Health Status Check
        elif (df['notable_type'][i] in client_health_status ) or \
            (df['record_type'][i] in client_health_status ) or \
            (df['nature_of_incident'][i] in client_health_status ) or \
            (df['reason'][i] in client_health_status ):
            notable_type.append([df['notableId'][i], "Health Status Check"])
        #Step 12: Anything left over put in the 'Other' category
        else:
            notable_type.append([df['notableId'][i], "Other"])
    
    df_out = df.merge(pd.DataFrame(notable_type, columns = ['notableId', 'grp_nt_type']), left_on='notableId', right_on='notableId', how='left')
    df_out.date = pd.to_datetime(df_out.date, utc=True)
    return df_out

In [19]:
rawfile_location_notables = os.path.join(dataset_folder, 'notables_prod.csv')
rawfile_location_control_notables = os.path.join(dataset_folder, 'control_notables_prod.csv')
    
wci_column_drop = [
    'last_modified_date_time', 'area', 'action_taken', 'time_of', 'status', 'created_timestamp', 
    'updated_timestamp', 'updated_author', 'created_author']

control_column_drop = ['last_modified_date_time', 'area', 'time_of', 'status', 'place_of_incident',
                       'date_of_notification', 'action_taken', 'other_type',
                       'prior_level_of_care', 'new_level_of_care']

df_notable_in = pd.read_csv(rawfile_location_notables)
df_notable_in.drop(columns=wci_column_drop, inplace=True)
df_notable_in.rename(columns= {'client_id':'clientId', 'date_of':'date', 'notable_id':'notableId'}, inplace=True)

df_control_notable_in = pd.read_csv(rawfile_location_control_notables)
df_control_notable_in.drop(columns=control_column_drop, inplace=True)
df_control_notable_in.rename(columns= {'client_id':'clientId', 'date_of':'date', 'notable_id':'notableId'}, inplace=True)

df_notables = create_notable_groups(df_notable_in).reset_index(drop=True)
    
df_control_notables = create_notable_groups(df_control_notable_in).reset_index(drop=True)
    
df_notables['wci_control_client'] = False;
df_control_notables['wci_control_client'] = True;

df_notables = pd.concat([df_notables, df_control_notables])

df_notables = df_notables \
    .groupby(['clientId', 'grp_nt_type']).any().max(axis=1) \
    .reset_index('grp_nt_type') \
    .pivot(columns='grp_nt_type', values=0) \
    .fillna(False)

df_notables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419 entries, 0010A000003UCPIQA4 to 001U000001k5HVMIA2
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Account Status Change              419 non-null    bool 
 1   Care Plan/Schedule Update          419 non-null    bool 
 2   Client Issue/Incident/Complaint    419 non-null    bool 
 3   CoVID 19                           419 non-null    bool 
 4   Death                              419 non-null    bool 
 5   Health Decline                     419 non-null    bool 
 6   Health Status Check                419 non-null    bool 
 7   Hospitalization                    419 non-null    bool 
 8   Injury/Fall                        419 non-null    bool 
 9   LOC Change                         419 non-null    bool 
 10  Other                              419 non-null    bool 
 11  Potential LOC Change (Health)      419 non-null    bool 


In [13]:
import json
import ast

# HSPP Health Indicators
#
# In addition to the pitch-toolkit survey data, a set of basic health-related measures are recorded in each client visit. 
# These measures are part of Kindreds regular care, and pulled from the Kindred datasets using our 
# vsf-analytics::Migrate.java utility (i.e., a cron-style data migration).
#
# Health Indicators are stored as a column in the vsf-analytics::hspp_assessment table, *roughly* encoded as a JSON string 
# (see `_parse_encoded_health_indicators_string` utility for a caveat). These measures are captured during *every* visit, 
# and may therefore change over time (e.g., a patient may acquire a new assistive device such as a cane or walker).
#
# This module exposes a single `get_health_indicators` utility function that parses, processes, and sanitizes these health 
# indicators for all clients in the dataset. Specifically, this function returns a `two-tuple` of pandas data frames, 
# formatted as follows:
#
#   (per_client_health_indicators, per_visit_health_indicators)
#    
# ...where:
#
#   - `per_client_health_indicators` - A data frame with a single row for each *client* in the dataset. This data frame 
#     will include clients from both the `control` and `wci` groups. For clients with more than one set of recorded health 
#     indicators in the dataset, this data frame will contain the *most recently reported* values.
# 
#   - `per_visit_health_indicators` - A data frame with a row for each *visit* in the dataset. Note however, that only 
#     `wci` group clients will have more than a single occurence in the data frame.
#
# TODO: This utility function reports a `DataFrame is highly fragmented` Performance Warning because of the way I've
# incrementally built up the sanitized/processed columns in the original dataframe. I believe that the warning is telling
# us that there is no guarantee that the entire df will occupy a single contiguous chunck of memory, which could hurt the
# performance of some native numpy routines (i.e., that expect to operate on contiguous arrays).
#
# The fix for this is probably to rework the private utilities in this module to each return separate dataframes, and then
# concatenate all of these together for the final return. However, until performance is actually a concern, I think we are
# fine.

def get_health_indicators(dataset_folder_name):
    per_visit_indicators = _import_health_indicators(dataset_folder_name)
    
    _create_onehot_mental_columns(per_visit_indicators)
    _create_onehot_physical_columns(per_visit_indicators)
    _create_onehot_assistive_devices_columns(per_visit_indicators)
    _create_boolean_smoker_column(per_visit_indicators)
    _create_boolean_pets_column(per_visit_indicators)
    _create_simplified_house_condition_column(per_visit_indicators)
    _convert_to_friendly_dates(per_visit_indicators)
    
    per_client_indicators = per_visit_indicators.groupby('client_id').first()
    
    return per_client_indicators



# Importing the Health Indicators Dataset
#
# Health indicators are stored as a string-encoded object in the `health_indicators` field of each visit record. 
# We can transform this into a valid python object using the `parse_health_indicators` utility define above.
# For now, I'm throwing away the outer `visit` fields, as these are also available within the list of health indicators.
def _import_health_indicators(dataset_folder_name):
    control_group_filename = 'control_assessment_Data.csv'
    wci_group_filename = 'assessment_Data.csv'
    
    control_group_health_indicators = _read_health_indicators_from_file(dataset_folder_name + '/' + control_group_filename)
    wci_group_health_indicators = _read_health_indicators_from_file(dataset_folder_name + '/' + wci_group_filename)
    
    # The `wci_control_client` health indicator field denotes whether this client was in the intervention or control group of the experiment. 
    # However, this field isn't populated for every record in this dataset (e.g., earlier visits before this column was added)
    # Therefore, explicitly assign a value for every record.
    control_group_health_indicators['wci_control_client'] = True
    wci_group_health_indicators['wci_control_client'] = False

    return pd.concat([control_group_health_indicators, wci_group_health_indicators], axis=0)


def _read_health_indicators_from_file(filename):
    visits = pd.read_csv(filename)
    
    # Each set of "health indicators" is encoded in the `health_indicators` column of the df.
    # First, we need to parse each string-encoded entry...
    visits.health_indicators = visits.health_indicators.map(_parse_encoded_health_indicators_string)

    # ...The health indicators are now represeted as a valid python object. However, this object is
    # still stored in a single column of the data frame.
    #
    # Thefore, "explode" this object and create a new data frame that represents each "health indicator" field as a separate column
    return pd.DataFrame(list(visits.health_indicators))


# Kindred records a set of "health indicators" for each of their client (e.g., demographics, health conditions, details of their care plan). 
# These "health indicators" are stored as a string-encoded field, however, the exact encoding method seems to differ between clients 
# (e.g., some seem to be encoded in JSON, while others use python literal dictionary format (e.g., the result of calling dict.repr()).
#
# This utility encapsulates these encoding format details by trying all possible expected encodings until one works. A 
# `HealthIndicatorsDecodeError` is raised when an unexpected formatting is encountered.
def _parse_encoded_health_indicators_string(health_indicators_string):
    try:
        return json.loads(health_indicators_string)
    except json.JSONDecodeError:
        pass
    
    try:
        return ast.literal_eval(health_indicators_string)
    except Exception:
        pass
    
    raise HealthIndicatorsDecodeError()
    
class HealthIndicatorsDecodeError(Exception):
    pass

    
# Metal Health Details - Transform to Onehot Fields
#
# We can convert the `mental_health_details` field to a series of onehot fields, each indicating
# the presence of the corresponding condition.
#
# TODO: Note that I've currently manually created this list of `mental_conditions` by visually inspecting
# the `value_counts()` output from the `mental_health_details` column - in the future we could programatically, 
# however I think this is fine.
def _create_onehot_mental_columns(health_indicators_df):
    mental_conditions = [
        'dementia', 
        "alzhemier's",
        'depression',
        'short-term memory impairment',
        'loss of orientation',
        'anxiety',
        'bi-polar', 
        'other'
    ]
    
    for condition in mental_conditions:
        def has_condition(condition_string):
            return condition in condition_string.lower()

        column_label = 'has_mental_' + condition.replace(' ', '_')
        health_indicators_df[column_label] = health_indicators_df.mental_health_details.map(lambda x: int(has_condition(x)), na_action='ignore')

    health_indicators_df['mental_count'] = health_indicators_df.filter(regex='^has_mental_').sum(axis=1)


# Physical Health Details - Transform to Onehot Fields
#
# Similar to the `mental_health_details` field, however, this appears to be a free-form text
# data field as opposed to choice dropdown. Looks like employees were instructed to use a
# semi-colon separate list later in the program, however still pretty variable formatting.
def _create_onehot_physical_columns(health_indicators_df):
    physical_conditions = [
        'diabetes', 
        'stroke', 
        'high blood pressure', 
        'cancer', 
        'copd', 
        'arthritis', 
        'high cholesterol', 
        'mobility issues', 
        'vision impairment', 
        'hearing impairment',
        'chronic pain',
        'gastro-intestinal issue',
        'heart failure',
        'other'
    ]

    for condition in physical_conditions:
        def has_condition(conditions_string):
            return condition in conditions_string.lower()

        column_label = 'has_physical_' + condition.replace(' ', '_')
        health_indicators_df[column_label] = health_indicators_df.health_physical_issues_details.map(lambda x: int(has_condition(x)), na_action='ignore')


    health_indicators_df['physical_count'] = health_indicators_df.filter(regex='^has_physical_').sum(axis=1)

# Special Equipment Details - Transform to Onehot fields
#
# Again, this `special_equipment_details` columns seems to be a free-text field. Transform
# into a series of onehot fields using a keyword-matching approach. The list of keywords was
# determined by visually inspecting the `value_counts()` output.
def _create_onehot_assistive_devices_columns(health_indicators_df):
    mobility_assistance_devices = [
        'cane',
        'walker',
        'wheelchair',
        'tub',
        'toilet',
        'commode',
        'catheter',
        'grab bar',
        'scooter',
        'rollator',
        'crutches',
        'bed railing',
        'shower chair',
        'hospital bed',
        'lifeline',
    ]

    for device in mobility_assistance_devices:
        def has_device(device_string):
            return device in device_string.lower()

        column_label = 'has_mobility_' + device.replace(' ', '_')
        health_indicators_df[column_label] = health_indicators_df.special_equipment_details.map(lambda x: int(has_device(x)), na_action='ignore')

    health_indicators_df['mobility_count'] = health_indicators_df.filter(regex='^has_mobility_').sum(axis=1)


# Smoker
#
# This column is currently categorical -- let's collapse into a boolean column...
#   - True <= 'Client smokes outside'
#   - True <= 'Client smokes inside'
#   - False <= 'Client is a non-smoker'
def _create_boolean_smoker_column(health_indicators_df):
    health_indicators_df['is_smoker'] = health_indicators_df.client_smoker.map(lambda s: int('smokes' in s), na_action='ignore')


# Pets
#
# Lets collapse this into a boolean column indicating presence of *any* pet
def _create_boolean_pets_column(health_indicators_df):
    pet_keywords = ['dog', 'cat', 'other']

    def has_pets(pet_string):
        return any([keyword in pet_string.lower() for keyword in pet_keywords])

    health_indicators_df['has_pets'] = health_indicators_df.pets_details.map(lambda x: int(has_pets(x)), na_action='ignore')


# House Condition
#
# This category appears to be a free-text field...
# This is a pretty good apprach, but note that it does mis-categories some entries (e.g., "house is not tidy")
def _create_simplified_house_condition_column(health_indicators_df):
    label_keywords = {
        'clean': ['tidy', 'acceptable', 'clean', 'good', 'well kept' 'excellent', 'spotless', 'immaculate', 'impeccable', 'organized'],
        'dirty': ['cluttered', 'crowded', 'rough', 'poor', 'disrepair', 'messy']
    }

    def parse_house_condition(condition_string):
        lowered = condition_string.lower()
        for (label, keywords) in label_keywords.items():
            for keyword in keywords:
                if keyword in lowered:
                    return label
        return 'n/a'

    health_indicators_df['house_condition'] = health_indicators_df.condition_of_client_s_house.map(parse_house_condition, na_action='ignore')


# Convert string-dates to date objects for easier plotting
def _convert_to_friendly_dates(health_indicators_df):
    for column in ['assessment_date', 'service_start_date']:
        health_indicators_df[column] = pd.to_datetime(health_indicators_df[column])

In [18]:
client_indicators = get_health_indicators(dataset_folder)
health_indicators = client_indicators.merge(df_notables, left_on='client_id', right_on='clientId')
health_indicators.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419 entries, 0 to 418
Columns: 149 entries, weight to Potential LOC Change (Non-Health)
dtypes: bool(38), datetime64[ns](2), float64(56), int64(5), object(48)
memory usage: 382.2+ KB


In [15]:
health_indicators.to_csv("health_indicators_fall_detection.csv", sep=',', header=True)